# Assignment 6.8 Model Building

Nikki Aaron (na5zn), Kevin Hoffman (keh4nb), Ashley Scurlock (ams5zx), Siddharth Surapaneni (sss2ea)

In [1]:
import pandas as pd
import os

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pandas as pd

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [2]:
from pyspark.ml.regression import LinearRegression

# Load training data
df = spark.read.csv("data_combined.csv", inferSchema =True, header=True)
df.printSchema()

root
 |-- zip: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- timezone: string (nullable = true)
 |-- county_name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state_id: string (nullable = true)
 |-- state_name: string (nullable = true)
 |-- population: double (nullable = true)
 |-- density: double (nullable = true)
 |-- res_rate: double (nullable = true)
 |-- annual_mbtu_used: double (nullable = true)
 |-- annual_kwh_used: double (nullable = true)
 |-- temp_Apr: double (nullable = true)
 |-- temp_Aug: double (nullable = true)
 |-- temp_Dec: double (nullable = true)
 |-- temp_Feb: double (nullable = true)
 |-- temp_Jan: double (nullable = true)
 |-- temp_Jul: double (nullable = true)
 |-- temp_Jun: double (nullable = true)
 |-- temp_Mar: double (nullable = true)
 |-- temp_May: double (nullable = true)
 |-- temp_Nov: double (nullable = true)
 |-- temp_Oct: double (nullable = true)
 |-- temp_Sep: double (nullable 

In [3]:
# Select columns for training
label = "dollars_saved"

feature_to_ignore = ["res_rate", "annual_output_w_hrs", "anuual_output_kwh20_sps", "percent_current_needs_met", "country_name", "county_name", "city", "state_id", "state_name", "timezone"]
features = [c for c in df.columns if c not in feature_to_ignore + [label]]

df_clean = df.select([c for c in df.columns if c not in feature_to_ignore])
df_train, df_test = df_clean.randomSplit([0.8, 0.2], 500)
df_train.take(1)

[Row(zip=1002, lat=42.37492, lng=-72.4621, population=30014.0, density=210.5, annual_mbtu_used=63.0, annual_kwh_used=18463.47741, temp_Apr=61.05, temp_Aug=80.35, temp_Dec=36.25, temp_Feb=22.95, temp_Jan=30.8, temp_Jul=85.25, temp_Jun=80.6, temp_Mar=39.9, temp_May=64.9, temp_Nov=45.75, temp_Oct=56.2, temp_Sep=79.1, pct_cloudy_days_Apr=70.85, pct_cloudy_days_Aug=63.3, pct_cloudy_days_Dec=69.15, pct_cloudy_days_Feb=67.9, pct_cloudy_days_Jan=64.3, pct_cloudy_days_Jul=63.85, pct_cloudy_days_Jun=67.0, pct_cloudy_days_Mar=73.9, pct_cloudy_days_May=68.80000000000001, pct_cloudy_days_Nov=73.6, pct_cloudy_days_Oct=66.05, pct_cloudy_days_Sep=63.9, dni=4.470000075000001, annual_output_kwh_20_sps=3042.215795183656, dollars_saved=516.1589089424086)]

## Model 1: Linear Regression
Best hyperparameters: maxIter:100, regParam:0, elasticNetParam: 0

Model Size: 148 KB

RMSE: 65.9148

In [4]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StandardScaler
from pyspark.ml import Pipeline 

assembler_features = VectorAssembler(
    inputCols=features,
    outputCol="features")

scaler = StandardScaler(inputCol="features", outputCol="scaled_features")

lr = LinearRegression(maxIter=100, labelCol=label, featuresCol="scaled_features", regParam=0.0, elasticNetParam=0.0,)

# Build the pipeline
pipeline = Pipeline(stages=[assembler_features, scaler, lr])

# Fit the pipeline
model = pipeline.fit(df_train)
#model.save("model-regression")
# Make a prediction
prediction = model.transform(df_test)


In [8]:
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Evaluate the model on training data
prediction.select(["dollars_saved", "prediction"]).show()

evaluator = RegressionEvaluator(
    labelCol=label, predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(prediction)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

+-----------------+------------------+
|    dollars_saved|        prediction|
+-----------------+------------------+
|484.5031021125742| 474.8286683709772|
|484.6682391251149|456.72195690845274|
|516.8557923471669|465.00491251832386|
|516.0491487817686|449.00953778037854|
|516.0173890931666|  435.398548433369|
|515.7661149761013| 479.3164287258172|
|515.8825341130485|474.49442316161293|
| 515.864331864375|434.12566696947437|
|516.4802310303666| 487.3332139283924|
|515.7580085695923| 434.7944581590876|
|484.4272456962049|473.09527872285526|
|516.0233877856182|435.57289012456744|
|516.4444948751209| 443.1325149456666|
|516.3146217883103| 435.1499450895241|
|516.8782180531416| 435.4562057109809|
|516.9301723033138| 434.8421577557409|
|516.8058037183525| 435.6214462503029|
|516.5456508321224|426.73458868118234|
|516.3220107680502|445.34295413300896|
|516.4141263941766|445.61465167269284|
+-----------------+------------------+
only showing top 20 rows

Root Mean Squared Error (RMSE) on test

## Model 2: Random Forest Regression
Best hyperparameters: maxDepth: 8, numTrees: 250, minInstancesPerNode: 1

Model Size: 4.9 MB

RMSE: 33.744

In [9]:
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor(featuresCol="scaled_features", labelCol=label, maxDepth=8, numTrees=250, minInstancesPerNode=1)

# Chain indexer and forest in a Pipeline
pipeline_rf = Pipeline(stages=[assembler_features, scaler, rf])

model = pipeline_rf.fit(df_train)
model.save("model-rf")
predictions_rf = model.transform(df_test)

evaluator = RegressionEvaluator(
   labelCol=label, predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions_rf)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 33.744


In [10]:
predictions_rf.select(["dollars_saved", "prediction"]).show()

+-----------------+-----------------+
|    dollars_saved|       prediction|
+-----------------+-----------------+
|484.5031021125742| 516.505343590026|
|484.6682391251149|517.4366364473047|
|516.8557923471669|518.5480942673072|
|516.0491487817686|518.4635486310632|
|516.0173890931666|518.0718040583994|
|515.7661149761013|516.8476291907117|
|515.8825341130485|516.8290975153025|
| 515.864331864375|518.1099441179163|
|516.4802310303666|517.9523654638882|
|515.7580085695923|518.1099441179163|
|484.4272456962049| 516.254079042995|
|516.0233877856182| 518.061149261007|
|516.4444948751209|518.2245319764656|
|516.3146217883103|518.1129351300027|
|516.8782180531416|517.0463894341927|
|516.9301723033138|513.7632972381936|
|516.8058037183525|513.6393711245371|
|516.5456508321224|519.4899607344098|
|516.3220107680502|519.1714144267758|
|516.4141263941766|519.1675502918422|
+-----------------+-----------------+
only showing top 20 rows



## Model 3: Gradient Boosted Trees
Best hyperparameters: maxDepth:8, minInstancesPerNode:2, maxIter: 100, stepSize: 0.1

Model Size: 1.7MB

RMSE: 20.7213

In [5]:
from pyspark.ml.regression import GBTRegressor

gbt = GBTRegressor(labelCol=label, predictionCol='prediction', featuresCol='scaled_features', maxDepth=8, minInstancesPerNode=2, maxIter=100, stepSize=0.1)

# Chain indexer and forest in a Pipeline
pipeline_gbt = Pipeline(stages=[assembler_features, scaler, gbt])

# Train model.  This also runs the indexer.
model = pipeline_gbt.fit(df_train)
model.save("model-gbt")
# Make predictions.
predictions_gbt = model.transform(df_test)


In [6]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(
    labelCol=label, predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions_gbt)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 20.7213


In [7]:
predictions_gbt.select(["dollars_saved", "prediction"]).show()

+-----------------+------------------+
|    dollars_saved|        prediction|
+-----------------+------------------+
|484.5031021125742| 506.8656149623041|
|484.6682391251149| 517.2272117653333|
|516.8557923471669| 513.6508706464688|
|516.0491487817686| 518.2075095279591|
|516.0173890931666| 516.7011605913955|
|515.7661149761013|510.68275608126334|
|515.8825341130485|511.71881025903707|
| 515.864331864375| 516.6866065674952|
|516.4802310303666| 512.2948439893455|
|515.7580085695923| 517.1193681228697|
|484.4272456962049| 507.3258785651495|
|516.0233877856182| 516.0678735473616|
|516.4444948751209| 515.7743016470491|
|516.3146217883103| 516.7414089283822|
|516.8782180531416| 513.5438512305753|
|516.9301723033138| 513.9171039350417|
|516.8058037183525| 514.1072459065782|
|516.5456508321224| 515.8038068640749|
|516.3220107680502|  520.957962688935|
|516.4141263941766| 520.8632931932017|
+-----------------+------------------+
only showing top 20 rows



In [9]:
# Save notebook as PDF document
!jupyter nbconvert --to pdf `pwd`/*.ipynb

[NbConvertApp] Converting notebook /sfs/qumulo/qhome/keh4nb/ds5559_project/Solar Models.ipynb to pdf
[NbConvertApp] Writing 42284 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', './notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', './notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 50631 bytes to /sfs/qumulo/qhome/keh4nb/ds5559_project/Solar Models.pdf
[NbConvertApp] Converting notebook /sfs/qumulo/qhome/keh4nb/ds5559_project/Solar Regression-Copy1.ipynb to pdf
[NbConvertApp] Writing 59218 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', './notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', './notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF succes